<a href="https://colab.research.google.com/github/applezhao/DeepRelated/blob/master/dataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preprossing (It should be executed only once!!!!!!!)

In [3]:
from google.colab import drive
drive.mount('/content/dgrive')

Mounted at /content/dgrive


**1. Mask tranparent pixels with red color**


In [0]:
#!pip install -U -q PIL
import os
import shutil
import random, os, time
from PIL import Image
def genInput(imageOriDIr, imageName, targetDir):
  image = Image.open(os.path.join(imageOriDIr, imageName))
  #save(os.path.join(targetDir, imageName))
  pixeldata = list(image.getdata())

  for i,pixel in enumerate(pixeldata):
    if pixel[3] < 200:
      pixeldata[i] = (255,0,0,pixel[3])

  image.putdata(pixeldata)
  image.save(os.path.join(targetDir, imageName))



for  root, dirs, files in os.walk(r'/content/dgrive/My Drive/colabData/originalData') :
  for name in files:
    imagePath = os.path.join(root, name)
    #print imagePath
    genInput(root, name, r'/content/dgrive/My Drive/colabData/redMaskedData')

**2. Crop / resize / flip image.**

In [0]:
import os
import shutil
import random, os, time
from PIL import Image
nameIndex = 0
def cropImage(imagePath, imageInutPath, tilesPerImage, targetW, targetH, targetDir):
    global nameIndex
    imOri = Image.open(imagePath)
    imInput = Image.open(imageInutPath)
    print imagePath
    size = 256, 256
    for i in range(1, tilesPerImage + 1):
        print nameIndex
        w, h = imOri.size
        x = random.randint(0, w - targetW - 1)
        y = random.randint(0, h - targetH - 1)
        print (x, y, x+targetW, y+targetH, w, h)
        oriCropped = imOri.crop((x, y, x+targetW, y+targetH))
        oriCropped.thumbnail(size, Image.ANTIALIAS)
        oriCropped.save(os.path.join(targetDir, "output", str(nameIndex)+".png"))
        imCropped = imInput.crop((x, y, x+targetW, y+targetH))
        imCropped.thumbnail(size, Image.ANTIALIAS)
        imCropped.save(os.path.join(targetDir, "input", str(nameIndex)+".png"))
        nameIndex += 1

def flipImage(imagePath, imageInutPath, tilesPerImage, targetW, targetH, targetDir):
    global nameIndex
    imOri = Image.open(imagePath)
    imInput = Image.open(imageInutPath)

    size = targetW, targetH
    imOri = imOri.transpose(Image.FLIP_LEFT_RIGHT)
    imOri.thumbnail(size, Image.ANTIALIAS)
    imOri.save(os.path.join(targetDir, "output", str(nameIndex)+".png"))
    imInput = imInput.transpose(Image.FLIP_LEFT_RIGHT)
    imInput.thumbnail(size, Image.ANTIALIAS)
    imInput.save(os.path.join(targetDir, "input", str(nameIndex)+".png"))
    nameIndex += 1

def resizeImage(imagePath, imageInutPath, tilesPerImage, targetW, targetH, targetDir):
    global nameIndex
    imOri = Image.open(imagePath)
    imInput = Image.open(imageInutPath)

    size = targetW, targetH
    imOri.thumbnail(size, Image.ANTIALIAS)
    imOri.save(os.path.join(targetDir, "output", str(nameIndex)+".png"))
    imInput.thumbnail(size, Image.ANTIALIAS)
    imInput.save(os.path.join(targetDir, "input", str(nameIndex)+".png"))
    nameIndex += 1

for  root, dirs, files in os.walk('/content/dgrive/My Drive/colabData/originalData') :
    for name in files:
        imageOriPath = os.path.join(root, name)
        imageInutPath = os.path.join('/content/dgrive/My Drive/colabData/redMaskedData', name)
        cropImage(imageOriPath, imageInutPath, 2, 800, 800, '/content/dgrive/My Drive/colabData/augmentedData')
        resizeImage(imageOriPath, imageInutPath, 30, 256, 256, '/content/dgrive/My Drive/colabData/augmentedData')
        flipImage(imageOriPath, imageInutPath, 30, 256, 256, '/content/dgrive/My Drive/colabData/augmentedData')

**3. Combine image to 512*256.**

In [0]:
import os
import shutil
import random, os, time
from PIL import Image

def concatImage(imageLeftPath, imageRightPath, name, targetDir):
    imLeft = Image.open(imageLeftPath)
    imRight = Image.open(imageRightPath)
    w, h = imLeft.size

    new_im = Image.new('RGBA', (w*2, h))
    new_im.paste(imLeft, (0,0))
    new_im.paste(imRight, (w,0))
    new_im.save(os.path.join(targetDir, name))

for  root, dirs, files in os.walk('/content/dgrive/My Drive/colabData/augmentedData/output') :
    for name in files:
        imageLeftPath = os.path.join(root, name)
        imageRightPath = os.path.join('/content/dgrive/My Drive/colabData/augmentedData/input', name)
        concatImage(imageLeftPath, imageRightPath, name, '/content/dgrive/My Drive/colabData/combinationData')

**4. Split train and val.**

In [23]:
import os
import shutil
import random, os, time
from PIL import Image

def splitDataset(fullfileList, targetDir):
  i = 0
  for each in fullfileList :
    orifile = each
    targetfile = os.path.join(targetDir, str(i)+".png")
    shutil.copyfile(orifile, targetfile)
    i+=1
      

allImageFiles = []
for  root, dirs, files in os.walk('/content/dgrive/My Drive/colabData/combinationData') :
    for name in files:
        allImageFiles.append(os.path.join(root, name))
random.shuffle(allImageFiles)

train = allImageFiles[:int((len(allImageFiles)+1)*.80)]
val = allImageFiles[int((len(allImageFiles)+1)*.80):]
print len(train)
print len(val)
print len(allImageFiles)
splitDataset(train, '/content/dgrive/My Drive/colabData/train')
splitDataset(val, '/content/dgrive/My Drive/colabData/val')

202
50
252
